In [17]:
import os
import dspy
import random
from dotenv import load_dotenv
from dspy.clients.lm_local import LocalProvider
from dspy.datasets import DataLoader
from datasets import load_dataset
from typing import Literal


In [4]:
load_dotenv()  # Load environment variables from .env file

True

## Setup MLFlow

In [2]:
import mlflow

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("DSPy")

2025/09/24 21:25:22 INFO mlflow.tracking.fluent: Experiment with name 'DSPy' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/602383626347086788', creation_time=1758767122300, experiment_id='602383626347086788', last_update_time=1758767122300, lifecycle_stage='active', name='DSPy', tags={}>

## Load the Dataset

In [13]:
# Load the Banking77 dataset.
CLASSES = load_dataset("PolyAI/banking77", split="train", trust_remote_code=True).features['label'].names
kwargs = dict(fields=("text", "label"), input_keys=("text",), split="train", trust_remote_code=True)

# Load the first 2000 examples from the dataset, and assign a hint to each *training* example.
raw_data = [
    dspy.Example(x, label=CLASSES[x.label]).with_inputs("text")
    for x in DataLoader().from_huggingface(dataset_name="PolyAI/banking77", **kwargs)[:1000]
]

random.Random(0).shuffle(raw_data)

In [14]:
len(CLASSES), CLASSES[:10]

(77,
 ['activate_my_card',
  'age_limit',
  'apple_pay_or_google_pay',
  'atm_support',
  'automatic_top_up',
  'balance_not_updated_after_bank_transfer',
  'balance_not_updated_after_cheque_or_cash_deposit',
  'beneficiary_not_allowed',
  'cancel_transfer',
  'card_about_to_expire'])

In [15]:
unlabeled_trainset = [dspy.Example(text=x.text).with_inputs("text") for x in raw_data[:500]]

unlabeled_trainset[0]

Example({'text': 'What if there is an error on the exchange rate?'}) (input_keys={'text'})

## Create the Classifier

In [16]:
classify = dspy.ChainOfThought(f"text -> label: Literal{CLASSES}")

In [ ]:
student_lm_name = "meta-llama/Llama-3.2-1B-Instruct"
student_lm = dspy.LM(model=f"openai/local:{student_lm_name}", provider=LocalProvider(), max_tokens=2000)
teacher_lm = dspy.LM('openai/gpt-5-mini', max_tokens=3000)

In [33]:
from dspy.clients.lm_local import LocalProvider

student_lm_name = "meta-llama/Llama-3.2-1B-Instruct"
student_lm = dspy.LM(model=f"openai/local:{student_lm_name}", provider=LocalProvider(), max_tokens=2000)
teacher_lm = dspy.LM('gemini/gemini-2.5-flash-lite', max_tokens=3000)

In [34]:
student_classify = classify.deepcopy()
student_classify.set_lm(student_lm)

teacher_classify = classify.deepcopy()
teacher_classify.set_lm(teacher_lm)

In [35]:
dspy.settings.experimental = True  # fine-tuning is an experimental feature, so we set a flag to enable it

optimizer = dspy.BootstrapFinetune(num_threads=16)  # if you *do* have labels, pass metric=your_metric here!
classify_ft = optimizer.compile(student_classify, teacher=teacher_classify, trainset=unlabeled_trainset)

2025/09/24 22:21:31 INFO dspy.teleprompt.bootstrap_finetune: Preparing the student and teacher programs...
2025/09/24 22:21:31 INFO dspy.teleprompt.bootstrap_finetune: Bootstrapping data...


Average Metric: 499.00 / 499 (100.0%): 100%|█████████▉| 498/500 [00:15<00:00, 30.50it/s]

2025/09/24 22:21:49 WARNING dspy.clients.lm: LM response was truncated due to exceeding max_tokens=3000. You can inspect the latest LM interactions with `dspy.inspect_history()`. To avoid truncation, consider passing a larger max_tokens when setting up dspy.LM. You may also consider increasing the temperature (currently 0.0)  if the reason for truncation is repetition.


Average Metric: 500.00 / 500 (100.0%): 100%|██████████| 500/500 [00:19<00:00, 26.10it/s]

2025/09/24 22:21:50 INFO dspy.evaluate.evaluate: Average Metric: 500 / 500 (100.0%)
2025/09/24 22:21:50 INFO dspy.teleprompt.bootstrap_finetune: Preparing the train data...
2025/09/24 22:21:50 INFO dspy.teleprompt.bootstrap_finetune: Using 500 data points for fine-tuning the model: openai/local:meta-llama/Llama-3.2-1B-Instruct
2025/09/24 22:21:50 INFO dspy.teleprompt.bootstrap_finetune: Starting LM fine-tuning...
2025/09/24 22:21:50 INFO dspy.teleprompt.bootstrap_finetune: 1 fine-tuning job(s) to start
2025/09/24 22:21:50 INFO dspy.teleprompt.bootstrap_finetune: Starting 1 fine-tuning job(s)...
2025/09/24 22:21:50 INFO dspy.teleprompt.bootstrap_finetune: Calling lm.kill() on the LM to be fine-tuned to free up resources. This won't have any effect if the LM is not running.


ImportError: cannot import name 'terminate_process' from 'sglang.utils' (/Users/williamthompson/Code/projects/dspy-entity-extraction/.venv/lib/python3.13/site-packages/sglang/utils.py)